<img style="float:left" src="images\python_logo.png" alt="python Logo" height="80" width="150">


# <center> PIR SAP-Excel Process Automation </center>



### <u> Key Libraries used </u>
<b> For Excel & Data Cleansing </b>
- pandas
- xlwings (or you can use xlsxwriter)
- difflib
- numpy

```python
!pip freeze  # OR use !pip list to check all your existing libraries
```

<b> For SAP </b>

- win32com.client
- sys
- subprocess
- os
- getpass
- time


In [2]:
import pandas as pd
import xlwings as xw
import xlsxwriter
from difflib import get_close_matches
import datetime
import numpy as np
import win32com.client
import sys
import subprocess
import os
import getpass
import time###  All of the codes below allow multi-condition row-level filtering

In [ ]:
os.system("TASKKILL /F /IM saplogon.exe")  # this will kill SAP if necessary

try:
    xw.apps.active.quit()  # This will kill Excel if necessary
except:
    pass

In [ ]:
homedir = os.path.expanduser("~")+'\Desktop' # Getting desktop directory
username = getpass.getuser()   # This is the user name


df_daily = pd.read_excel("Freight PIR_Upload Template.xlsx", sheet_name = "Sheet1" , header=None)

In [ ]:
##### Step C: Entering basic PIR Data
def enter_data(session, df):

    

    for i in range(df.shape[0]):
        print(i)
              
        ### Step C1: Exectue T-code ME12
        session.findById("wnd[0]/tbar[0]/okcd").text = "me12"
        session.findById("wnd[0]").sendVKey(0)
        
        ### Step C2: Input data from PIR file
        print("============================================")
        print("Vendor is " , df['Vendor'].iloc[i])

        
        session.findById("wnd[0]/usr/ctxtEINA-LIFNR").text = df['Vendor'].iloc[i] # Vendor
#             time.sleep(0.5)
        session.findById("wnd[0]").sendVKey(0)  # Enter Key
#             time.sleep(0.5)
        session.findById("wnd[0]").sendVKey(8)  # F8 Key to click Conditions
#             time.sleep(0.5)      
            
        
        ### Step D: Exit to main SAP page
        session.findById("wnd[0]").sendVKey(15) # Shift + F3 Key to exit


    ### Step E Write SAP data onto upload template
    wb = xw.Book('Freight PIR_Upload Template.xlsx')
    sht_PIR = wb.sheets['Sheet1']
    # Pasting into Cell A1  whatever data you grabbed from SAP
    sht_PIR.range('A' + str(1)).options(index=False, header=False).value = df_sap_data      


In [ ]:
def sap_automate(df):          # This function will Login to SAP and perform T-code

    try:
        ##### Step A: Opening SAP
        path = r"C:\Program Files (x86)\SAP\FrontEnd\SAPgui\saplogon.exe"
        subprocess.Popen(path) # This opens SAP
        time.sleep(4)  # deliberate time delay for application to open

        SapGuiAuto = win32com.client.GetObject('SAPGUI')
        if not type(SapGuiAuto) == win32com.client.CDispatch:
            return

        application = SapGuiAuto.GetScriptingEngine
        if not type(application) == win32com.client.CDispatch:
            SapGuiAuto = None
            return
        connection = application.OpenConnection("XXX-XXXX-XXX-XXXX", True)

        if not type(connection) == win32com.client.CDispatch:
            application = None
            SapGuiAuto = None
            return

        session = connection.Children(0)
        if not type(session) == win32com.client.CDispatch:
            connection = None
            application = None
            SapGuiAuto = None
            return
        
        ##### Step B: Logging into your account
        session.findById("wnd[0]/usr/txtRSYST-BNAME").text = "MY_ID"  # USER ID
        session.findById("wnd[0]/usr/pwdRSYST-BCODE").text = "MY_PASSWORD" # PASSWORD
        session.findById("wnd[0]").sendVKey(0)
        session.findById("wnd[0]").sendVKey(0) # Additional enter key to get past compliance msg
        
        
        ##### Step C: Entering Data
        enter_data(session=session, df = df)  # I use this function to execute needed steps

        
    except:
        
        print(sys.exc_info()[0])

    finally:
        session = None
        connection = None
        application = None
        SapGuiAuto = None
 
sap_automate(df_daily)  